In [ ]:
frame - supervised: as the target value(CO2) emmision is provided as label
        regression: we are predicting CO2 emmission which is numerical
        batch: as of now, we have no plan to make this model to be trained with future incomming new data